# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import string
import re
import nltk
import string
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
nltk.download(['punkt', 'wordnet', 'stopwords'])
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
import sklearn
print(sklearn.__version__)

0.19.1


### 2. Split data into X and Y 

In [4]:
# load data from database
con = 'sqlite:///disaster_db.db'
engine = create_engine(con)
df = pd.read_sql_table('messages',con)
X = df['message']
Y = df.iloc[:,4:]
category_names = Y.columns.tolist()

In [7]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Additional data analysis to Plot graphs in webapp

In [47]:
def get_categ_cnt(df,col_list,op_cat_colname, op_num_colname):
    '''
    INPUT 
        df - a dataframe schema name
        col_list - a Python list with all column names on which count has to be done 
        op_cat_colname - category column name
        op_num_colname - count column name (should be int)
    OUTPUT
        df_result - a sorted data farme with each category name and count of the columns
    '''
    result_dict = {}
    for each_col in col_list:
        result_dict[each_col] = df[each_col].sum()
    
    df_result = pd.DataFrame(pd.Series(result_dict)).reset_index()
    df_result.columns = [op_cat_colname, op_num_colname]
    return df_result.sort_values(op_num_colname, ascending=False)

col_list=['floods', 'storm', 'fire', 'earthquake', 'cold','other_weather']
weather_related = ( df["weather_related"] == 1)
df_weather = df[weather_related]
weather_data =  get_categ_cnt(df_weather,col_list,'Weather_Category','count')  
weather_data

,Weather_Category,count
3,earthquake,2452
1,storm,2440
0,floods,2149
5,other_weather,1376
4,cold,528
2,fire,282


In [46]:
weather_related_death = ( df["death"] == 1 ) & ( df["weather_related"]==1 )
df_new = df[weather_related_death]
weather_death =  get_categ_cnt(df_new,col_list,'Weather_Category','Death_count') 
weather_death


,Weather_Category,Death_count
0,floods,335
3,earthquake,260
1,storm,238
5,other_weather,166
4,cold,63
2,fire,33


In [43]:
df_weather_death

,index,0
0,floods,335
1,storm,238
2,fire,33
3,earthquake,260
4,cold,63
5,other_weather,166


In [49]:
weather_death.Weather_Category.tolist()

['floods', 'earthquake', 'storm', 'other_weather', 'cold', 'fire']

In [50]:
list(weather_death.Death_count)

[335, 260, 238, 166, 63, 33]

In [61]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)
graph_two = []
graph_two.append(
      go.Bar(
      x = weather_death.Weather_Category.tolist(),
      y = weather_death.Death_count.tolist(),
      )
    )
layout_two = dict(title = 'Weather Related Death',
                xaxis = dict(title = 'Weather Condition',),
                yaxis = dict(title = 'Count'),
                )
fig = go.Figure(data=graph_two, layout = layout_two)
iplot(graph_two)

In [63]:
weather_data

,Weather_Category,count
3,earthquake,2452
1,storm,2440
0,floods,2149
5,other_weather,1376
4,cold,528
2,fire,282


In [65]:

graph_three = []
weather_cat = weather_data['Weather_Category'].tolist()
count = weather_data['count'].tolist()
graph_three.append(go.Pie(
                    labels = weather_cat, 
                    values = count)
                  )
fig = go.Figure(data = graph_three)
iplot(fig)

In [60]:

graphs = []
graphs.append(dict(data=graph_two, layout=layout_two))

### Create reusable functions to be used in the pipeline

#### Tokenization function to process text

In [11]:
def tokenize(text):
    """Tokenization function. 
    Input : Raw text 
    Output: Returns tokenized text. After normalized, stop words removed, stemmed and lemmatized.
    """
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    stop_words = nltk.corpus.stopwords.words("english")
    # Remove punctuations, change to lower case
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    # tokenize text
    tokens = nltk.word_tokenize(text)
    # lemmatize and remove stop words
    return [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]

#### A function to display classification report for each column

In [12]:
Y.related.value_counts()

1    19874
0     6115
2      188
Name: related, dtype: int64

In [13]:
# We've found some data cleanup required in predictions 
Y.related.replace(2, 0, inplace=True)
Y.related.value_counts()

1    19874
0     6303
Name: related, dtype: int64

In [28]:
# to speed up calc
#X = X.head(50)
#Y = Y.head(50)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [15]:
clf = RandomForestClassifier()
pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(clf))
                    ])

In [16]:
print(pipeline)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=22)

In [21]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(19632,)
(6545,)
(19632, 36)
(6545, 36)


In [22]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
y_pred = pipeline.predict(X_test)
y_pred_train = pipeline.predict(X_train)

In [43]:
print(classification_report(Y_test.values, y_pred, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.86      0.90      0.88      5030
               request       0.78      0.40      0.53      1112
                 offer       0.00      0.00      0.00        34
           aid_related       0.73      0.59      0.66      2690
          medical_help       0.56      0.11      0.18       538
      medical_products       0.76      0.08      0.15       310
     search_and_rescue       0.59      0.06      0.10       177
              security       0.00      0.00      0.00       124
              military       0.80      0.07      0.13       226
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.26      0.40       400
                  food       0.86      0.47      0.60       710
               shelter       0.79      0.34      0.47       579
              clothing       0.71      0.10      0.17       103
                 money       0.78      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [44]:
# Print classification report on training data
print('\n',classification_report(Y_train.values, y_pred_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.99      0.99      0.99     14844
               request       1.00      0.93      0.96      3352
                 offer       1.00      0.75      0.86        83
           aid_related       1.00      0.97      0.98      8150
          medical_help       1.00      0.86      0.92      1542
      medical_products       1.00      0.86      0.93      1000
     search_and_rescue       1.00      0.78      0.87       547
              security       0.99      0.72      0.84       347
              military       1.00      0.86      0.93       633
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.90      0.95      1269
                  food       1.00      0.94      0.97      2207
               shelter       1.00      0.92      0.96      1729
              clothing       1.00      0.85      0.92       301
                 money       1.00    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [47]:
clf = RandomForestClassifier()

pipeline1 = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(clf))
                    ])
param_grid = {
        'clf__estimator__n_estimators': [10,20],
        'clf__estimator__min_samples_split': [2]  
      #,  'clf__estimator__criterion': ['entropy', 'gini']
     }

In [48]:
cv = GridSearchCV(pipeline1, param_grid=param_grid,scoring='f1_micro',n_jobs=-1,verbose=20)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.5888800925643993, total=  54.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.592601703514652, total=  54.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.6043403666134907, total=  55.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.6168635486981677, total= 1.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  7.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.6161861133566873, total= 1.6min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  8.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  8.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=20)

In [50]:
# Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

In [52]:

# Print classification report on test data
print(classification_report(Y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.95      0.98      0.97      3983
               request       0.98      0.84      0.90       892
                 offer       1.00      0.65      0.79        17
           aid_related       0.94      0.91      0.93      2149
          medical_help       0.99      0.72      0.83       404
      medical_products       0.98      0.73      0.84       252
     search_and_rescue       0.98      0.68      0.81       149
              security       0.98      0.57      0.72        84
              military       0.99      0.72      0.84       173
           child_alone       0.00      0.00      0.00         0
                 water       0.97      0.82      0.89       344
                  food       0.98      0.86      0.92       604
               shelter       0.96      0.82      0.88       458
              clothing       0.96      0.75      0.84        72
                 money       0.99      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [53]:
from pprint import pprint
pprint(cv.best_estimator_.steps)

[('vect',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f20202dee18>, vocabulary=None)),
 ('tfidf',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
 ('clf',
  MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1

In [54]:
def evaluate_model(model, X_test, Y_test, category_names):
  
    Y_pred = model.predict(X_test)
    
    # Calculate model accuracy 
    for i in range(len(category_names)):
        print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
        print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [55]:
evaluate_model(cv, X_test, Y_test, category_names)

Category: related 
              precision    recall  f1-score   support

          0       0.94      0.85      0.89      1253
          1       0.95      0.98      0.97      3983

avg / total       0.95      0.95      0.95      5236

Accuracy of                   related: 0.95
Category: request 
              precision    recall  f1-score   support

          0       0.97      1.00      0.98      4344
          1       0.98      0.84      0.90       892

avg / total       0.97      0.97      0.97      5236

Accuracy of                   request: 0.97
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5219
          1       1.00      0.65      0.79        17

avg / total       1.00      1.00      1.00      5236

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.94      0.96      0.95      3087
          1       0.94      0.91  

In [ ]:
#evaluate_model(cv.best_estimator_, X_test, Y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [57]:
import joblib
joblib.dump(cv.best_estimator_, 'disaster_model_udacity_pj2.pkl')

['disaster_model_udacity_pj2.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [58]:
model = joblib.load("disaster_model_udacity_pj2.pkl")

In [59]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f20202dee18>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
               oob_score=False, random_state=None,